<a href="https://colab.research.google.com/github/tdiffendal/USAT/blob/master/census-responses/census_response_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from collections import OrderedDict

In [ ]:
numbersasnumbers = [
    "CR20",
    "CR10",
    "CDIFF",
    "internet",
    "population",
    "households",
    "white"
]

In [ ]:
highlightfields = [
    "county_name",
    "CR20",
    "CDIFF",
    "minoritypct",
    "internetpct"
]

In [ ]:
fieldsets = [
# human name, field name, sortorder reversed?
    ("high return rate", "CR20", True),
    ("low return rate", "CR20", False),
    ("high population", "population", True)
]

In [ ]:
countydict = {}
with open("counties.csv", "r") as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        state = row['State']
        if state not in countydict:
            countydict[state] = []
        fips = row['FIPS20']
        del row['']    # Lose pandas row number
        for thingy in numbersasnumbers:
            if row[thingy] != '':
                row[thingy] = float(row[thingy])
        row['minority'] = ''
        row['minoritypct'] = ''
        row['internetpct'] = ''
        if row['white'] != '':
            if row['population'] != '' and row['population'] != 0:
                row['minoritypct'] = 100 * (row['population'] - row['white'])/row['population']
            if row['households'] != '' and row['households'] != 0:
                row['internetpct'] = 100 * row['internet'] / row['households']
        countydict[state].append(row)

In [ ]:
#

In [ ]:
statedict = {}
with open("states.csv", "r") as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        del row['']    # Lose pandas row number
        state = row['State']
        statedict[state] = row

In [ ]:
for state in statedict:
    if state not in countydict:
        print(state)
for state in countydict:
    if state not in statedict:
        print(state)
# OK, nothing extra, nothing missing

In [ ]:
# statedict.keys()

In [ ]:
# PR and DC! Yay!

In [ ]:
# https://stackoverflow.com/questions/13573123/sort-dictionary-of-dictionaries-by-value
sr20ranklist = sorted(statedict,key=lambda state:statedict[state]['SR20'], reverse=True)
len(sr20ranklist)

52

In [ ]:
for state in statedict:
    statedict[state]["sr20rank"] = sr20ranklist.index(state) + 1    
    for fieldset in fieldsets:   #         # human name, field name, sortorder reversed?
        humanname, fieldname, sortorder = fieldset
        locallist = [d for d in countydict[state] if d[fieldname] != '']   # Filter out empty entries
        templist = sorted(locallist, key = lambda i: i[fieldname], reverse=sortorder)   # Sort
        # OK, DC trips us up by not having more than one county.
        for i in range(1, 3 + 1):
            for highlightfield in highlightfields:
                statedict[state][f"{humanname}{i} {highlightfield}"] = ""   # Store a blank entry for each
                if len(templist) >= i:
                    statedict[state][f"{humanname}{i} {highlightfield}"] = templist[i-1][highlightfield]

In [ ]:
headers = list(statedict['Florida'].keys())
with open("census-response-wide.csv", "w", newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(headers)
    for state in sorted(statedict):
        writer.writerow(list(statedict[state].values()))